In [177]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import string
import re
import joblib
import os

In [178]:
reviews = pd.read_csv("reviewsClean.csv")

In [179]:
#All lower case
#Remove numbers
#Remove punctuation
#translator = str.maketrans('', '', string.punctuation)
#Remove whitespace

# for i in range(len(reviews)):
#     reviews["reviews"][i] = reviews["reviews"][i].lower()
#     reviews["reviews"][i] = re.sub(r'\d+', '', reviews["reviews"][i])
#     reviews["reviews"][i] = reviews["reviews"][i].translate(translator)
#     reviews["reviews"][i] = " ".join(reviews["reviews"][i].split())
# #print(reviews)
# reviews.to_csv("reviewsClean.csv", index=False)


In [180]:
#print(len(reviews["reviews"][0].split()))
#print(round(sum([len(i.split())for i in reviews["reviews"]])/len(reviews["reviews"])))

max_vocab_length = 10000
max_length = 10469

In [181]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length,
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(reviews["reviews"])

In [182]:
#random_sentence = random.choice(reviews["reviews"])
#print(random_sentence)
#print(text_vectorizer(random_sentence))

#words_in_vocab = text_vectorizer.get_vocabulary()
#top_5_words = words_in_vocab[:5]
#bottom_5_words = words_in_vocab[-5:]
#print(top_5_words,bottom_5_words)

In [183]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=64,
                             input_length=max_length)


#print(embedding(text_vectorizer(random_sentence)))

In [184]:
pooling_layer = layers.GlobalAveragePooling1D()

In [185]:
x = text_vectorizer(reviews["reviews"])
x = embedding(x)
x = pooling_layer(x)

In [318]:
nn = NearestNeighbors(n_neighbors=24,algorithm='ball_tree',p=2)
nn.fit(x)

NearestNeighbors(algorithm='ball_tree', n_neighbors=24)

In [319]:
# Save model
directory = "saved_model"

if not os.path.exists(directory):
    os.makedirs(directory)

# Define the file path
file_path = os.path.join(directory, "nearest_neighbors.joblib")
joblib.dump(nn, file_path)

['saved_model\\nearest_neighbors.joblib']

In [320]:
# Load model
if  os.path.exists(directory):
    nn = joblib.load(file_path)

In [321]:
text = embedding(text_vectorizer("family"))
neighbours = nn.kneighbors(text, return_distance=False)

In [322]:
for index in neighbours[0]:
    movie_id = reviews.iloc[index]["imdb_id"]
    print(f"Movie ID: {movie_id}")

Movie ID: tt2404311
Movie ID: tt0449089
Movie ID: tt1698641
Movie ID: tt3289712
Movie ID: tt1772288
Movie ID: tt0763831
Movie ID: tt1620981
Movie ID: tt1408253
Movie ID: tt2059171
Movie ID: tt11327514
Movie ID: tt0119528
Movie ID: tt3832914
Movie ID: tt0252028
Movie ID: tt1294970
Movie ID: tt10342730
Movie ID: tt2279339
Movie ID: tt1137450
Movie ID: tt0765476
Movie ID: tt0108550
Movie ID: tt1567437
Movie ID: tt1187064
Movie ID: tt0093010
Movie ID: tt0086154
Movie ID: tt0115798
